<a href="https://colab.research.google.com/github/jouvetg/igm/blob/main/notebooks/IGM_aletsch_1880_2100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <h1 align="center" id="title">IGM notebook tutorial (aletsch-1880-2100) </h1>

This set-up aims to closely reproduce the simulations of the Great Aletsch Glacier (Switzerland) in the past and in the future based
on the CH2018 climate scenarios described in (Jouvet and al., JOG, 2011) and (Jouvet and Huss, JOG, 2019), respectively. 
The goal of this set-up is to give a template of climate-based mass balance implementation within the TensorFlow framework,
which can serve for future improvements of the Aletsch case, or adaptations to other mountain glaciers. This simulation relies on
an accumulation and distributed temperature-index melt mass balance model (Hock, JOG, 1999). The closest and most recent
description of the implemented model is given in (Jouvet and al., TC, 2020). This simulation does not stricly match the original
implementation due to discrepencies in input data and ice flow models (emulated Stokes vs original Stokes). Therefore, melt
factors were re-calibrated to get the best match between observed and modelled top ice surface in the past. 


Let us firt download the IGM code, ice flow emulator, and input data. Input files include geological inputs (geology.nc), spatially-varying fields for the computation of melt factors and snow reditribution (massbalance.nc), as well as temperature and precipitation time serie data (temp_prec.dat). The data also provide some observed top surface ice topographies (in 1880, 1926, 1957, 1980, 1999, 2009, 2017 and 2016).

In [ ]:
# get the core source of igm (in igm.py)
!wget -O igm.py https://raw.githubusercontent.com/jouvetg/igm/main/src/igm.py

# get input data (topography, mass balance and climate data, resp.)
!wget -O geology.nc https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/geology.nc
!wget -O massbalance.nc https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/massbalance.nc
!wget -O mbparameter.dat https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/mbparameter.dat
!wget -O temp_prec.dat https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/temp_prec.dat

# get specific igm functions for this set-up (climate and mass balance, resp.)
!wget -O igm_clim_aletsch.py https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/igm_clim_aletsch.py
!wget -O igm_smb_accmelt.py https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/igm_smb_accmelt.py

# get the ice flow DL-emulator
!apt install subversion
!svn export https://github.com/jouvetg/igm/trunk/model-lib/f12_cfsflow_GJ_21_a

# force version 2.4 of Tensorflow
%tensorflow_version 2.4

First, we import necessary libraries, check the version of tensorflow, and inport the class igm (defined in igm.py), and create an igm object.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import sys
print('TF version is ',tf.__version__)
sys.argv = ['']  # this is absolutly necessary in Jupyter notebook

from igm import *
from igm_clim_aletsch import *
from igm_smb_accmelt import *
 
# add extensions for climate generation and mass balance to the core igm class
class igm(igm,igm_clim_aletsch, igm_smb_accmelt):
    pass

# define the igm class
igm = igm()

Altough, you don't have to, it is highly recommended to run under GPU (click above 'Runtime' -> 'Change Runtime type' -> Activate GPU). Then the fowllowing comand check that a GPU is indeed available, and give the name of the GPU. (With the free veryion of colab these GPU are often old and not that efficient -- like the K80 --, IGM would typically run faster on new material)

Next, we overide the default configuration parameters, initialize the class, and check at the parameters

In [ ]:
# Set-up parameters

igm.config.working_dir           = ''  
igm.config.tstart                = 1880  # starting time (you may use other available year like 1926)
igm.config.tend                  = 2100  # final time
igm.config.tsave                 = 1     # saving frequence
igm.config.init_strflowctrl      = 78    # this parametrizes the ice flow, the higher this param, the faster the flow
igm.config.cfl                   = 0.25  # CFL condition must be lower than 1

igm.config.iceflow_model_lib_path= 'f12_cfsflow_GJ_21_a'  # direct to the iceflow emaultor
igm.config.type_climate          = 'aletsch'              # select type of climate forcing

igm.config.type_mass_balance     = 'accmelt'              # select type of mass balance
igm.config.massbalance_file      = 'massbalance.nc'       # mass balance data
igm.config.weight_accumulation   = 1.00                   # this param weights accumulation
igm.config.weight_ablation       = 1.25                   # this param weights ablation

igm.config.usegpu                = True

igm.config.weight_Aletschfirn    = 1.0 # this param weights Aletschfirn accumulation area
igm.config.weight_Jungfraufirn   = 1.0 # this param weights Jungfraufirn accumulation area
igm.config.weight_Ewigschneefeld = 1.0 # this param weights Ewigschneefeld accumulation area

igm.initialize()

We are now ready to simulate the great Aletsch Glacier for 220 years from 1880. The next code could be substitute by the simple comand igm.run(), however, we use the full workflow as we included an assemement of the model output ice surface against observation in years 1880, 1926, 1957, 1980, 1999, 2009, 2017 and 2016.


In [ ]:
with tf.device(igm.device_name):

    igm.load_ncdf_data(igm.config.geology_file)
    
    # load the surface toporgaphy available at given year
    igm.usurf.assign(vars(igm)['surf_'+str(int(igm.t))])
    igm.thk.assign(igm.usurf-igm.topg)
    
    igm.initialize_fields()
    igm.initialize_iceflow()
    igm.update_climate()
    igm.update_smb()
    igm.update_iceflow()
    igm.update_ncdf_ex()
    igm.update_ncdf_ts()
    igm.print_info()

    while igm.t < igm.config.tend:
        
        igm.tcomp["All"].append(time.time())
             
        # For thes year, check the std between modelled and observed surfaces
        if igm.t in [1926,1957,1980,1999,2009,2017]:
            diff = (igm.usurf-vars(igm)['surf_'+str(int(igm.t))]).numpy()
            diff = diff[igm.thk>1]
            mean  = np.mean(diff)
            std   = np.std(diff)
            vol   = np.sum(igm.thk) * (igm.dx ** 2) / 10 ** 9
            print(" Check modelled vs observed surface at time : %8.0f ; Mean discr. : %8.2f  ;  Std : %8.2f |  Ice volume : %8.2f " \
                  % (igm.t, mean, std, vol) )
        
        igm.update_climate()
        igm.update_smb() 
        igm.update_iceflow()
        igm.update_t_dt()
        igm.update_thk()
        igm.update_ncdf_ex()
        igm.update_ncdf_ts()
        igm.update_plot()
        igm.print_info()
        
        igm.tcomp["All"][-1] -= time.time()
        igm.tcomp["All"][-1] *= -1
        
    igm.print_all_comp_info()

We have now modelled 220 years of evolution of the Great Aletsch Glacier at 100-meter resolution in about one minute. IGM monitors key variables (ice volume, time step) during computation, and provide STDs between modelled and observed ice surfaces in some years. Melt factors were tuned to keep this STD fairly small (below 35 m). The next code permits to vizualize the simulation.


In [ ]:
from IPython.display import HTML, display
import xarray as xr 
from matplotlib import pyplot as plt, animation

# open the thickness variable from the netcdf file (choose ice thcikness or ice speed)
#var  = xr.open_dataset('ex.nc',engine='netcdf4').thk            ; maxvar = 800
var  = xr.open_dataset('ex.nc',engine='netcdf4').velbar_mag    ; maxvar = 250
 
fig, ax = plt.subplots(figsize=(8,10))

# Plot the initial frame. 
cax = var[0,:,:].plot(add_colorbar=True,cmap=plt.cm.get_cmap('jet', 10),vmin=0, vmax=maxvar)
ax.axis("off") ; ax.axis("equal")

# dont' show the original frame
plt.close()

def animate(i):
    cax.set_array(var[i,:,:].values.flatten())
    ax.set_title("Time = " + str(var.coords['time'].values[i])[:13])

ani = animation.FuncAnimation( fig, animate, frames=var.shape[0], interval=100 ) # interval in ms between frames

HTML(ani.to_html5_video())

# optionally the animation can be saved in avi
# ani.save('animation.mp4')

In [ ]:
# always good to clean the space
!rm ts.nc ex.nc igm-run-parameters.txt

Let's now change the mass balance moddel, and use a well-trained neural network trained from climate and mass balance data (from Aletsch Glacier). This can be seen as an alternative to the accumulation / melt model. Therefore, you must set igm.config.type_mass_balance to 'nn', provide a valid path for the mass balance emulator igm.config.smb_model_lib_path, and also force monthly climate inputs (instead of daily) setting igm.config.clim_time_resolution to 12.

In [ ]:
# get the SMB emulator
!svn export https://github.com/jouvetg/igm/trunk/model-lib/smb1_meteoswissglamos_GJ_21_a

# option 2: emulated smb model by a CNN -- uncoment these lines
igm.config.smb_model_lib_path    = 'smb1_meteoswissglamos_GJ_21_a'
igm.config.type_mass_balance     = 'nn'
igm.config.clim_time_resolution  = 12
igm.config.tstart                = 2017
igm.config.tend                  = 2100

Then simply launch IGM time evolution:

In [ ]:
igm.run()

The simulation shows slightly higher volumes (compared to before), but does not lead to substantially different results.

In [ ]:
# clean all
! rm -r f12_cfsflow_GJ_21_a smb1_meteoswissglamos_GJ_21_a __pycache__
! rm *.nc  *.py *.dat *.txt